## 实现反向传播（Back Propagation）

本节我们将做两个独立示例，一个回归例子，一个分类例子。

为阐释如何使用 tf 反向传播，我们先加载必要的库并重置计算图。

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()

### 创建图会话

In [2]:
sess = tf.Session()

### 回归示例
---

我们创建一个如下回归示例：输入数据为 100 个 normal 随机采样（mean: 1.0, stdev: 0.1）。目标值是 100 个 10.0

如此适配回归模型：x_data * A = target_values

理论上，我们知道 A 应该等于 10.0。

先创建数据和目标并 placeholder

In [4]:
x_vals = np.random.normal(1, 0.1, 100)
y_vals = np.repeat(10., 100)
x_data = tf.placeholder(shape=[1], dtype=tf.float32)
y_target = tf.placeholder(shape=[1], dtype=tf.float32)

现在创建用于计算图的变量，A

In [5]:
# 创建变量（一个模型变量=A）
A = tf.Variable(tf.random_normal(shape=[1]))

把模型操作加入图。下面只是把输入乘 A 已得到输出。

In [6]:
# 添加到计算图
my_output = tf.multiply(x_data, A)

接下来我们指定损耗函数。这将允许 tf 知道如何改变模型变量。我们使用 L2 损耗函数。注意：若要用 L1 损耗函数，把 `tf.square()` 改为 `tf.abs()`

In [7]:
# 添加 L2 损耗函数到计算图
loss = tf.square(my_output - y_target)

现在初始化变量，需要指出的是，这是使用随机标准正常数字在计算图上初始化变量A

In [8]:
# 初始化变量
init = tf.global_variables_initializer()
sess.run(init)

我们需要创建一个优化操作。这里我们使用标准的 `GradientDescentOptimizer()`，并告诉 tf 最小化损耗。这里使用 0.02 的学习率，但可在这个值附近随心所欲的尝试，最后可看到学习曲线。然而，请注意，学习率过大将导致算法不会收敛。

In [9]:
# 创建优化器
my_opt = tf.train.GradientDescentOptimizer(0.02)
train_step = my_opt.minimize(loss)

### 运行回归图

接下来迭代运行计算图 100 次，没迭代 25 次打印 A 和 损耗值。我们将看到 A 值越来越趋近 10，同时损耗降低。

In [11]:
# 循环运行
for i in range(100):
    rand_index = np.random.choice(100)
    rand_x = [x_vals[rand_index]]
    rand_y = [y_vals[rand_index]]
    sess.run(train_step, feed_dict={x_data: rand_x, y_target: rand_y})
    if (i+1) % 25 == 0:
        print('Step #' + str(i+1) + ' A = ' + str(sess.run(A)))
        print('Loss = ' + str(sess.run(loss, feed_dict={x_data: rand_x, y_target: rand_y})))

Step #25 A = [ 9.92020321]
Loss = [ 1.03096735]
Step #50 A = [ 9.88349724]
Loss = [ 0.3415648]
Step #75 A = [ 9.77454758]
Loss = [ 0.04842026]
Step #100 A = [ 10.07893467]
Loss = [ 4.997334]


## 分类例子
---

